In [ ]:
%pip install seaborn
%pip install ipympl

In [ ]:
%matplotlib widget

In [ ]:
import contextlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
import PIL

In [ ]:
# helper function for loading input images
async def load_img(name):
    from js import fetch
    import pickle
    import io

    URL = f"https://raw.githubusercontent.com/sk1p/diffractogram-analysis/main/content/{name}"

    res = await fetch(URL)
    buffer = await res.arrayBuffer() 

    data = buffer.to_py()

    data_buf = io.BytesIO(data)
    return data_buf

In [ ]:
rings_buf = await load_img("data/rings.jpg")
with PIL.Image.open(rings_buf) as im:
    rings = np.asarray(im)

In [ ]:
sns.set()

# Step 1: Calibrate the image

Pick the pixel positions between identical gold diffraction
spots on opposite sides of the origin of the digital diffractogram. This defines the center and scale of the diffractogram.

Hover the mouse over the image that is plotted with the cell below. It should show the position in pixel coordinates and the RGB value under the cursor.

The cell below should look like this:

```python
spot_pair = np.array([
    (y1, x1),
    (y2, x2)
]
```

In [ ]:
fig, axes = plt.subplots()
axes.imshow(rings)

In [ ]:
spot_pair = np.array([
    (98, 256), # y1, x1
    (210, 92),  # y2, x2
])

## Validation

In [ ]:
fig, axes = plt.subplots()
axes.imshow(rings)
for (y, x) in spot_pair:
    axes.scatter(x, y)

## Calculate spot distance, radius and center

In [ ]:
distance = np.linalg.norm(spot_pair[1] - spot_pair[0])
radius = distance / 2
center = np.mean(spot_pair, axis=0)
distance, radius, center

# Step 2: enter your measured points

Enter successive rows of `n` and (y, x) pixel coordinates on the ring-shaped minima and maxima corresponding to `n`. Even values for `n` are minima and odd values maxima of the ring pattern. You can enter several rows with pixel coordinates for the same `n` to improve the accuracy.

By making use of prior knowledge that the image was recorded underfocus, successive rings correspond to values of `n` of -2, -4, -6, etc.

The cell below should look like this:

```python
measured_points = [
    (n1, (y1, x1)),
    (n2, (y2, x2)),
    (n3, (y3, x3)),
    (n4, (y4, x4)),
    # etc.
]
```

In [ ]:
fig, axes = plt.subplots()
axes.imshow(rings)

In [ ]:
measured_points = [
    # insert your measurements here:
    # (2, (157, 208)),
    # (2, (163, 142)),
    # (3, (137, 211)),
    # ...
    
    (-2, (157, 208)),
    (-2, (163, 142)),
    (-3, (137, 211)),
    (-10, (107, 244)),
]

At least two points with different `n` are required for fitting!

In [ ]:
if len(measured_points) < 2:
    raise RuntimeError("Please update `measured_points` above! There have to be at least two (n, u) rows!")

# Step 3: Check the measured points

Confirm that all coordinates are entered correctly. Red points should be placed on maxima and blue points on minima

In [ ]:
def is_odd(x):
    # bitwise and
    return x & 1

fig, axes = plt.subplots()
axes.imshow(rings)
for i, (n, (y, x)) in enumerate(measured_points):
    color = 'red' if is_odd(n) else 'blue'
    axes.scatter(x, y, c=color)
    axes.annotate(f'#{i}: n={n}', (x + 6, y + 3), c=color)

# Step 4: Convert pixel coordinates to physical coordinates

Use the calibration and the known value for gold to convert the position of the rings to distance from the center in nm$^{-1}$

In [ ]:
ring_distances = [(n, np.linalg.norm((y, x) - center)) for (n, (y, x)) in measured_points]

In [ ]:
gold_spacing = 1/0.235  # unit: nm^-1

In [ ]:
ring_spacing = [(n, u_pixel / radius * gold_spacing) for (n, u_pixel) in ring_distances]

# Step 5: calculate u^2 and n/u^2

In [ ]:
calculated = [
    (n, u, u**2, n/(u**2))
    for (n, u) in ring_spacing
]

In [ ]:
df = pd.DataFrame(calculated, columns=["n", "u", "u^2", "n/u^2"])
df

# Step 6: Linear fit and plot

In [ ]:
lin = LinearRegression()
lin.fit(df[['u^2']].values, df['n/u^2'].values)

ax = df.plot.scatter(x='u^2', y='n/u^2')
# Evaluate the fit at the intercept and at the maximum for plotting the fit line
x_points = np.array([0, np.max(df['u^2'])])
ax.plot(x_points, lin.predict(x_points[:, np.newaxis]), c='r')

# Step 6: Extract slope and intercept from the fit

In [ ]:
lin.intercept_

In [ ]:
lin.coef_

# Step 7: Determine $\Delta f$ and Cs

In [ ]:
lamb = 0.00251  # unit: nm

In [ ]:
Df = lin.intercept_ / 2 /lamb
Df

In [ ]:
Cs = lin.coef_[0] / lamb**3
Cs